In [2]:
from pint import UnitRegistry


In [3]:
u = UnitRegistry()
ft = u.feet
inc = u.inches

In [10]:
class ComponentReportClass:
    def __init__(self, depth, length, width):
        self.depth = depth
        self.length = length
        self.width = width 
    # dimensional calcs   
    def calc_volume(self):
        return (self.depth).to("yards") * (self.width).to("yards") * (self.length).to("yards")
    def calc_top_face_area(self):
        return self.length * self.width 
    def calc_sides_face_area(self):
        return 2 * (self.depth * self.width ) + 2 * (self.depth * self.length )
    # conversion calcs
    def to_cubic_yards(a):
        return a/27

    # basic work calcs
    def calc_subgrade_prep(self):
        bottom = self.calc_top_face_area()
        sides = self.calc_sides_face_area()
        return bottom + sides
    def calc_formwork(self):
        return self.calc_sides_face_area()
    def calc_excavation(self):
        # TODO add benching...
        return self.calc_volume()
    def calc_concrete(self):
        return self.calc_volume()* 1.08

    


    # reports
    def report_basic(self):
        report = {
            "subgrade": self.calc_subgrade_prep(),
            "formwork": self.calc_formwork(),
            "excavation": self.calc_excavation(),
            "concrete": self.calc_concrete()
        }
        return report 


    

In [30]:
def calc_component_totals(components, fx):
    """components are defined components
    fx is a function that takes in a list of components """
    c_reports = [c.report_basic() for c in components]

    sum_components = {}
    for k in c_reports[0].keys():
        sum_components[k] = fx(c_reports, k)

    return sum_components

In [33]:
# pile caps 
pc1 = ComponentReportClass(2*ft, 4*ft, 4*ft) # reported in ft!

pc2 = ComponentReportClass(2*ft + 9*inc, 
                            17*ft + 2*inc, 
                            10*ft + 6*inc) 
def fx(list, key):
   return 2*list[0][key] + list[1][key]

calc_component_totals([pc1, pc2], fx)

{'subgrade': 428.4166666666667 <Unit('foot ** 2')>,
 'formwork': 216.16666666666669 <Unit('foot ** 2')>,
 'excavation': 20.729166666666664 <Unit('yard ** 3')>,
 'concrete': 22.3875 <Unit('yard ** 3')>}

In [41]:
# gradebeams
gb_length = 6 * (2*ft + (6*inc + (1/8)*inc))
gb_width = gb_height = (12*inc).to("feet")
gb1 = ComponentReportClass(gb_width, gb_length,  gb_height) # reported in ft!

gb1.report_basic()


# def fx(list, key):
#    return 2*list[0][key] + list[1][key]

# calc_component_totals([pc1, pc2], fx)

{'subgrade': 47.1875 <Unit('foot ** 2')>,
 'formwork': 32.125 <Unit('foot ** 2')>,
 'excavation': 0.5578703703703703 <Unit('yard ** 3')>,
 'concrete': 0.6025 <Unit('yard ** 3')>}

In [42]:
# all 
def fx(list, key):
    # 2* pc1 + 1*pc2 * 1*gb1
   return 2*list[0][key] + list[1][key] + list[2][key]

calc_component_totals([pc1, pc2, gb1], fx)

{'subgrade': 475.6041666666667 <Unit('foot ** 2')>,
 'formwork': 248.29166666666669 <Unit('foot ** 2')>,
 'excavation': 21.287037037037035 <Unit('yard ** 3')>,
 'concrete': 22.99 <Unit('yard ** 3')>}